# Gerador de Placas de Identificação 3D

Este notebook gera modelos 3D (.3mf) de placas de identificação personalizadas.

**Instruções:**
1. Execute a célula de instalação (se necessário).
2. Execute o código principal.
3. Siga as instruções para inserir nomes e escolher cores.
4. Escolha onde salvar os arquivos.

In [ ]:
# Instala dependências
%pip install build123d ipykernel

In [1]:
import tkinter as tk
from tkinter import filedialog
from build123d import Color
import os
import sys
import importlib

# Importa a biblioteca de utilitários local
if os.getcwd() not in sys.path:
    sys.path.append(os.getcwd())
    
try:
    import nameplate_utils
    importlib.reload(nameplate_utils)
    from nameplate_utils import generate_plate_model, export_plate
except ImportError:
    print("Erro: Não foi possível importar 'nameplate_utils.py'. Verifique se o arquivo está na mesma pasta.")

In [2]:
def get_user_inputs():
    print("--- Configuração das Placas ---")
    print("Dimensões padrão: 10cm x 5cm x 0.4cm")
    
    # 1. Quantidade
    while True:
        try:
            qtd = int(input("Quantas placas você deseja criar? (1-5): "))
            if 1 <= qtd <= 5:
                break
            print("Por favor, escolha entre 1 e 5.")
        except ValueError:
            print("Entrada inválida.")
    
    plates = []
    
    # 2. Nomes
    for i in range(qtd):
        print(f"\nPlaca {i+1}:")
        nome = input("Digite o Nome (destaque): ").strip()
        if not nome:
            print("O nome não pode ser vazio.")
            nome = input("Digite o Nome (destaque): ").strip()
            
        sobrenome = input("Digite o Sobrenome (opcional, abaixo): ").strip()
        plates.append({"nome": nome, "sobrenome": sobrenome})
    
    # 3. Cores
    print("\n--- Escolha de Cores ---")
    color_options = {
        "1": ("Azul (Base) e Branco (Contorno)", Color('blue'), Color('white')),
        "2": ("Vermelho (Base) e Amarelo (Contorno)", Color('red'), Color('yellow')),
        "3": ("Preto (Base) e Branco (Contorno)", Color('black'), Color('white')),
        "4": ("Verde (Base) e Preto (Contorno)", Color('green'), Color('black')),
        "5": ("Branco (Base) e Preto (Contorno)", Color('white'), Color('black')),
        "6": ("Laranja (Base) e Azul (Contorno)", Color('orange'), Color('blue'))
    }
    
    for key, val in color_options.items():
        print(f"{key} - {val[0]}")
        
    while True:
        choice = input("Escolha uma opção de cor (número): ")
        if choice in color_options:
            selected_colors = color_options[choice]
            break
        print("Opção inválida.")
        
    return plates, selected_colors[1], selected_colors[2]

def select_output_folder():
    print("\nSelecione a pasta para salvar os arquivos...")
    root = tk.Tk()
    root.withdraw()
    root.attributes('-topmost', True)
    folder = filedialog.askdirectory(title="Selecione onde salvar as placas")
    root.destroy()
    return folder

In [ ]:
def main():
    # Coletar dados
    plates_info, color_base, color_outline = get_user_inputs()
    
    # Selecionar pasta
    output_dir = select_output_folder()
    if not output_dir:
        print("Nenhuma pasta selecionada. Cancelando.")
        return
        
    print(f"\nGerando {len(plates_info)} placas em: {output_dir}...")
    
    for p in plates_info:
        n = p['nome']
        s = p['sobrenome']
        filename = f"{n}_{s}.3mf".replace(" ", "_")
        filepath = os.path.join(output_dir, filename)
        
        print(f"Gerando: {n} {s}...")
        try:
            # Gera o modelo
            model = generate_plate_model(n, s, color_base, color_outline)
            
            # Exporta
            export_plate(model, filepath)
            print(f"  -> Salvo: {filename}")
        except Exception as e:
            print(f"  ERRO ao gerar {n}: {e}")
            
    print("\nConcluído! Verifique a pasta de destino.")

if __name__ == "__main__":
    main()